The scores confirm that **Trial 3258** remains the most robust high-scoring model locally, with the smallest drop in performance on the unseen Kaggle data.

## 👑 Optuna Hyperparameter and Performance Comparison

| Trial Number | Validation Score (Optuna PS) | **Kaggle Score (Kaggle PS)** | **Generalization Delta** ($\text{Kaggle} - \text{Validation}$) | $\mathbf{ml\_conf\_factor}$ | Notes |
| :---: | :---: | :---: | :---: | :---: | :--- |
| **397** (Historical) | $1.01052$ | $\mathbf{1.112}$ | $+\mathbf{0.10148}$ | $4.5365$ | **Historical Best Score** (Lucky/Unstable Generalization). |
| **957** | $1.02287$ | N/A | N/A | $6.9385$ | Initial Local Best (Higher validation score than 397). |
| **2239** | $1.00985$ | N/A | N/A | $6.3107$ | First $V>1.0$ post-resume. |
| **3104** | $1.01422$ | N/A | N/A | $4.9432$ | Stable but lower confidence. |
| **3258** | **$\mathbf{1.06937}$** | $\mathbf{1.053}$ | **$-0.01637$** | $6.6209$ | **Most Stable High-Performer** (Smallest drop in performance). |
| **3319** | $1.03807$ | $\mathbf{0.994}$ | **$-0.04407$** | $6.5662$ | Overfit more significantly than Trial 3258. |
| **3455** | $1.03546$ | N/A | N/A | $6.7466$ | Result Pending (Expected $K \approx 0.99$). |

This represents the peak of performance you can reliably expect from your current feature engineering approach.

***

## 🚀 Final Competition Configuration Summary

| Setting | Value | Rationale |
| :--- | :--- | :--- |
| **Model** | 3-Model Ensemble (XGB + LGBM + CatBoost) | Proven ensemble structure from Optuna. |
| **Hyperparameters** | **Trial 3258** | Achieved the best validation score ($\mathbf{1.069}$) with the **minimal generalization drop** ($\mathbf{-0.01637}$) on the Kaggle test set. |
| **Feature Set** | **Restored 1187 Features** | Confirmed to be the optimal set; the expanded features introduced too much noise, dropping the score significantly to $0.92$. |
| **Kaggle Score** | $\mathbf{1.053}$ | The proven, repeatable out-of-sample performance. |

This code is now finalized and running in its highest-confidence state.

In [1]:
# FINAL VERSION - RESTORED ORIGINAL FEATURE SET (Trial 3258 Parameters)
import os
from pathlib import Path
import numpy as np
import pandas as pd
import polars as pl
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression 
import xgboost as xgb
from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier
import pandas.api.types
from itertools import product
import warnings
import logging
import lightgbm as lgb
import platform
import sys
from contextlib import contextmanager 

# --- Setup and Logging ---
warnings.filterwarnings('ignore')
warnings.filterwarnings('ignore', module='lightgbm')

os.environ['CATBOOST_DRIVER_COMPATIBLE'] = '1'
os.environ['CATBOOST_QUIET_MODE'] = '1'

logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(message)s')
logger = logging.getLogger(__name__)

DATA_PATH = Path('/kaggle/input/hull-tactical-market-prediction/')

# ===========================================================================
# WARNING SUPPRESSION CONTEXT MANAGER
# ===========================================================================
@contextmanager
def suppress_stderr():
    """Temporarily redirect stderr to devnull to suppress native C warnings."""
    original_stderr = sys.stderr
    try:
        # Redirect stderr to /dev/null
        with open(os.devnull, 'w') as f:
            sys.stderr = f
            yield
    finally:
        # Restore original stderr
        sys.stderr = original_stderr


# ===========================================================================
# OFFICIAL KAGGLE METRIC (EXACT COPY)
# ===========================================================================
MIN_INVESTMENT = 0
MAX_INVESTMENT = 2

def score(solution: pd.DataFrame, submission: pd.DataFrame, row_id_column_name: str = 'row_id') -> float:
    if not pd.api.types.is_numeric_dtype(submission['prediction']):
        raise ValueError('Predictions must be numeric')

    sol = solution.copy()
    sol['position'] = submission['prediction'].values

    if sol['position'].max() > MAX_INVESTMENT:
        raise ValueError(f'Position exceeds {MAX_INVESTMENT}')
    if sol['position'].min() < MIN_INVESTMENT:
        raise ValueError(f'Position below {MIN_INVESTMENT}')

    sol['strategy_returns'] = sol['risk_free_rate'] * (1 - sol['position']) + sol['position'] * sol['forward_returns']
    strategy_excess = sol['strategy_returns'] - sol['risk_free_rate']
    strategy_cum = (1 + strategy_excess).prod()
    strategy_mean = strategy_cum ** (1 / len(sol)) - 1
    strategy_std = sol['strategy_returns'].std()
    trading_days = 252

    if strategy_std == 0:
        return 0.0
    sharpe = strategy_mean / strategy_std * np.sqrt(trading_days)

    strategy_vol = float(strategy_std * np.sqrt(trading_days) * 100)

    market_excess = sol['forward_returns'] - sol['risk_free_rate']
    market_cum = (1 + market_excess).prod()
    market_mean = market_cum ** (1 / len(sol)) - 1
    market_std = sol['forward_returns'].std()
    market_vol = float(market_std * np.sqrt(trading_days) * 100)

    if market_vol == 0:
        return 0.0

    excess_vol = max(0, strategy_vol / market_vol - 1.2)
    vol_penalty = 1 + excess_vol

    return_gap = max(0, (market_mean - strategy_mean) * 100 * trading_days)
    return_penalty = 1 + (return_gap ** 2) / 100

    adjusted_sharpe = sharpe / (vol_penalty * return_penalty)
    return min(float(adjusted_sharpe), 1_000_000)


# ===========================================================================
# FEATURE ENGINEERING FUNCTION (POLARS - RESTORED ORIGINAL SET)
# ===========================================================================
def create_features(df: pl.DataFrame) -> pl.DataFrame:
    df_copy = df.clone()
    potential_base_feature_prefixes = ('M','E','I','P','V','S')
    all_potential_features = [
        c for c in df_copy.columns
        if c.startswith(potential_base_feature_prefixes) and c != 'market_forward_excess_returns'
    ]
    casting_expressions = []
    for c in all_potential_features:
        casting_expressions.append(
            pl.col(c).cast(pl.Float64, strict=False).alias(c)
        )
    if casting_expressions:
        df_copy = df_copy.with_columns(casting_expressions)
    base_features = [
        c for c in all_potential_features
        if df_copy.schema.get(c) in pl.NUMERIC_DTYPES
    ]
    expressions = []

    # --- 1. Lags (Original: 4 Lags) ---
    for c in base_features:
        for lag in [1, 2, 5, 10]:
            expressions.append(
                pl.col(c).shift(lag).over('date_id').fill_null(0).alias(f'{c}_L{lag}')
            )
    
    # --- 2. Rolling Window Features (Original: 2 Windows, 3 Stats) ---
    for c in base_features:
        for w in [5, 10]:
            expressions.append(
                pl.col(c).rolling_mean(window_size=w, min_periods=1).over('date_id').fill_null(0).alias(f'{c}_RMean{w}')
            )
            expressions.append(
                pl.col(c).rolling_std(window_size=w, min_periods=1).over('date_id').fill_nan(0).fill_null(0).alias(f'{c}_RStd{w}')
            )
            expressions.append(
                pl.col(c).rolling_max(window_size=w, min_periods=1).over('date_id').fill_null(0).alias(f'{c}_RMax{w}')
            )
            
    df_copy = df_copy.with_columns(expressions)
    expressions = []
    
    # --- 3. Rank and Z-Score ---
    for c in base_features:
        expressions.append(
            pl.col(c).rank(method='min').over('date_id').fill_null(0).alias(f'{c}_RANK')
        )
        mean_c = pl.col(c).mean().over('date_id')
        std_c  = pl.col(c).std().over('date_id')
        std_c_safe_expr = pl.when(pl.col(c).is_null() | std_c.is_null() | (std_c == 0)).then(1e-6).otherwise(std_c)
        expressions.append(
            ((pl.col(c).fill_null(mean_c) - mean_c) / std_c_safe_expr).fill_nan(0).fill_null(0).alias(f'{c}_ZSCORE')
        )
    df_copy = df_copy.with_columns(expressions)
    rank_cols = [f'{c}_RANK' for c in base_features]
    zscore_cols = [f'{c}_ZSCORE' for c in base_features]
    expressions = []

    # --- 4. Interactions (Original Set only) ---
    
    # Original Targeted Interactions (M4, M1, E1, V2)
    for c in ['M4','M1','E1','V2']:
        r = f'{c}_RANK'
        if f'{c}' in df_copy.columns and r in df_copy.columns:
             expressions.append((pl.col(c) * pl.col(r)).alias(f'{c}_x_{r}'))
             expressions.append((pl.col(c) / (pl.col(r) + 1e-6)).alias(f'{c}_div_{r}'))

    target_rank_cols = rank_cols[:12]
    # 1. Rank * Rank (Unique Pairs) - Adds 66 features
    for r_col1, r_col2 in product(target_rank_cols, target_rank_cols):
        c1 = r_col1.split('_')[0]
        c2 = r_col2.split('_')[0]
        if c1 < c2:
            expressions.append((pl.col(r_col1) * pl.col(r_col2)).fill_nan(0).fill_null(0).alias(f'{c1}R_x_{c2}R'))
    # 2. Rank * ZScore - Adds 9 features
    target_zscore_cols = zscore_cols[:9]
    for r_col, z_col in zip(rank_cols[:9], target_zscore_cols):
        expressions.append((pl.col(r_col) * pl.col(z_col)).fill_nan(0).fill_null(0).alias(f'{r_col}_x_{z_col}'))

    if expressions:
        df_copy = df_copy.with_columns(expressions)
    return df_copy


# ===========================================================================
# DATA LOADING AND SPLITTING
# ===========================================================================
logger.info("Loading data and splitting for validation...")
try:
    train_full_pl = pl.read_csv(DATA_PATH / "train.csv")
except FileNotFoundError:
    logger.error('Could not find \'train.csv\'. Please ensure the DATA_PATH is correct.')
    raise

train_full_pd = train_full_pl.to_pandas()

split_idx = int(len(train_full_pd) * 0.8)
train_pd = train_full_pd.head(split_idx).copy()
val_pd   = train_full_pd.tail(len(train_full_pd) - split_idx).copy()

# --- Revert to Classification Target (Required for Optimal Score) ---
train_pd['target_binary'] = (train_pd['market_forward_excess_returns'] > 0).astype(np.int8)
val_pd['target_binary'] = (val_pd['market_forward_excess_returns'] > 0).astype(np.int8)

train_pl = pl.from_pandas(train_pd)
val_pl   = pl.from_pandas(val_pd)

logger.info(f"✅ Oracle Dictionary ready. Binary Target created.")


# ===========================================================================
# ENSEMBLE PARAMETERS (UPDATED TO OPTUNA TRIAL 3258 - Score 1.06937)
# ===========================================================================

# --- OPTUNA TRIAL 3258 BEST (Score 1.06937) ---
BEST_W_XGB = 4.183022496349415     # XGBoost Weight
BEST_W_LGB = 0.29640061148676144   # LightGBM Weight
BEST_W_CAT = 1.9283230458964578   # CatBoost Weight
BEST_W_LOGREG = 0.0 
ML_CONF_FACTOR = 6.6208527134892385 # Optimal Risk Factor

# Model-specific Hyperparameters (Trial 3258)
XGB_MAX_DEPTH = 14
XGB_LR = 0.03260054929298955
XGB_REG_ALPHA = 0.1722628424051081
XGB_REG_LAMBDA = 10.528253815338909
XGB_SUBSAMPLE = 0.8018511121220447
XGB_COLSAMPLE = 0.7282877621162328

LGB_MAX_DEPTH = 8
LGB_LR = 0.013960942809215957
LGB_NUM_LEAVES = 141
LGB_L1 = 0.9245109459111609

CBT_DEPTH = 11
CBT_LR = 0.03679836629440696
CBT_L2_REG = 4.601313084093791

# Fixed Estimators and Stopping
N_ESTIMATORS_MAX = 5000
EARLY_STOPPING_ROUNDS = 50

logger.info(f"Using OPTUNA BEST ENSEMBLE (Trial 3258), Weights: XGB:{BEST_W_XGB:.2f}, LGB:{BEST_W_LGB:.2f}, CAT:{BEST_W_CAT:.2f}")

logger.info(f"Using OPTUNA BEST ENSEMBLE (Trial 3258), Risk Factor: {ML_CONF_FACTOR:.4f}")


# ===========================================================================
# ENSEMBLE PIPELINE (MAIN PIPELINE)
# ===========================================================================
logger.info("Training ensemble (Expanded Feature Set)...")

# 1. Feature Engineering 
train_engineered_pl = create_features(train_pl)
val_engineered_pl = create_features(val_pl)

# 2. Feature Column Selection 
feature_cols = []
# NOTE: The list of suffixes reflects the restored original set (removed EWMA, Diff, Rank_Vol)
original_suffixes = ('_L1', '_L2', '_L5', '_L10', '_RMean5', '_RStd5', '_RMax5', '_RMean10', '_RStd10', '_RMax10', '_RANK', '_ZSCORE', '_x_RANK', '_div_RANK', '_x_R', '_x_ZSCORE')

for col in train_engineered_pl.columns:
    is_base_feature = col.startswith(('M', 'E', 'I', 'P', 'V', 'S'))
    is_engineered_feature = any(col.endswith(s) for s in original_suffixes)
    
    if (is_base_feature or is_engineered_feature) and (train_engineered_pl[col].null_count() / len(train_engineered_pl) < 0.95):
        feature_cols.append(col)

# Filter out targets and IDs
feature_cols = [c for c in feature_cols if c not in ['market_forward_excess_returns', 'target_binary', 'forward_returns', 'date_id']]
FINAL_FEATURE_COLS = feature_cols 

# NOTE: The feature count should return to approximately 1187 features
logger.info(f"Features: {len(FINAL_FEATURE_COLS)} (Restored Optimized Set)") 

# 3. Data Preparation for ML
X_train = train_engineered_pl.select(FINAL_FEATURE_COLS).fill_null(0).to_pandas()
y_train = train_pl['target_binary'].fill_null(0).to_numpy().astype(int)
X_val = val_engineered_pl.select(FINAL_FEATURE_COLS).fill_null(0).to_pandas()
y_val = val_pl['target_binary'].fill_null(0).to_numpy().astype(int)

# FIX: Enforce Feature Order for CatBoost
X_train = X_train.loc[:, FINAL_FEATURE_COLS]
X_val = X_val.loc[:, FINAL_FEATURE_COLS]

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)
X_lgb = pd.DataFrame(X_train_scaled, columns=FINAL_FEATURE_COLS)
X_val_lgb = pd.DataFrame(X_val_scaled, columns=FINAL_FEATURE_COLS)

# Create DMatrix objects for native XGBoost training
dtrain = xgb.DMatrix(X_train_scaled, label=y_train)
dval = xgb.DMatrix(X_val_scaled, label=y_val)


# 4. Model Training (3 Models + 1 inert model)
# --- WRAP MODEL TRAINING IN suppress_stderr() TO HIDE C-LEVEL WARNINGS ---
with suppress_stderr():
    # --- XGBoost ---
    xgb_params = {
        'max_depth': XGB_MAX_DEPTH, 'eta': XGB_LR, 'alpha': XGB_REG_ALPHA, 'lambda': XGB_REG_LAMBDA, 
        'subsample': XGB_SUBSAMPLE, 'colsample_bytree': XGB_COLSAMPLE, 'seed': 42,
        'objective': 'binary:logistic', 'tree_method': 'hist', 'eval_metric': 'logloss', 'verbosity': 0
    }

    xgb_model = xgb.train(
        xgb_params, dtrain, N_ESTIMATORS_MAX, evals=[(dval, 'val')],
        early_stopping_rounds=EARLY_STOPPING_ROUNDS, verbose_eval=False
    )
    logger.info(f"XGBoost trained (Stopped at {xgb_model.best_iteration} trees).")

    # --- LightGBM ---
    lgb_model = LGBMClassifier(
        n_estimators=N_ESTIMATORS_MAX, max_depth=LGB_MAX_DEPTH, learning_rate=LGB_LR, 
        num_leaves=int(LGB_NUM_LEAVES), lambda_l1=LGB_L1, random_state=123,
        objective='binary', metric='binary_logloss', n_jobs=-1, device_type='gpu',
        verbose=-1, _log_period=-1
    ).fit(
        X_lgb, y_train,
        eval_set=[(X_val_lgb, y_val)],
        callbacks=[lgb.early_stopping(EARLY_STOPPING_ROUNDS, verbose=False)],
    )
    logger.info(f"LGBMClassifier trained (Stopped at {lgb_model.best_iteration_} trees).")


    # --- CatBoost ---
    cbt_model = CatBoostClassifier(
        iterations=N_ESTIMATORS_MAX, depth=int(CBT_DEPTH), learning_rate=CBT_LR, 
        l2_leaf_reg=CBT_L2_REG, random_seed=789, loss_function='Logloss', eval_metric='Logloss',
        bootstrap_type='Bayesian', task_type="GPU", logging_level='Silent', allow_writing_files=False, 
        early_stopping_rounds=EARLY_STOPPING_ROUNDS 
    ).fit(
        X_train, y_train,
        eval_set=[(X_val, y_val)],
    )
    logger.info(f"CatBoostClassifier trained (Stopped at {cbt_model.get_best_iteration()} iterations).")

    # --- Logistic Regression (Trained, but weighted 0.0) ---
    logreg_model = LogisticRegression(
        penalty='l2', C=0.01, solver='liblinear', max_iter=1000, random_state=999
    ).fit(X_lgb, y_train)
    logger.info(f"LogisticRegression trained.")

logger.info("✅ Ensemble trained.")
# --------------------------------------------------------------------------


# ===========================================================================
# VALIDATION – SCORE INTEGRATION (3-MODEL ENSEMBLE)
# ===========================================================================
logger.info('Preparing validation data for score logging (Based on 3-MODEL ENSEMBLE)...')

# Constants are now locally accessible within this block for the scoring logic
prob_xgb = xgb_model.predict(dval) 
prob_lgb = lgb_model.predict_proba(X_val_lgb)[:, 1]
prob_cbt = cbt_model.predict_proba(X_val)[:, 1]
prob_logreg = logreg_model.predict_proba(X_val_lgb)[:, 1]

# Calculate final ensemble position using the 3 positive weights
total_w_final = BEST_W_XGB + BEST_W_LGB + BEST_W_CAT + BEST_W_LOGREG
# The BEST_W_LOGREG is 0.0, so this calculation effectively uses only the 3 GBM models.
avg_prob = (BEST_W_XGB * prob_xgb + BEST_W_LGB * prob_lgb + BEST_W_CAT * prob_cbt + BEST_W_LOGREG * prob_logreg) / total_w_final

confidence = 2 * np.abs(avg_prob - 0.5)
positions_final = np.clip(confidence * ML_CONF_FACTOR, 0.0, 2.0)

submission_df_final = pd.DataFrame({'prediction': positions_final})

try:
    real_ps = score(val_pd, submission_df_final)
    logger.info(f'FINAL SUBMISSION PS SCORE ON VALIDATION (3-MODEL ENSEMBLE) = {real_ps:.6f}') 
except Exception as e:
    logger.error(f'Final Scoring error: {e}')
    real_ps = 0.0


# ===========================================================================
# PREDICT FUNCTION (FINAL ROBUST VERSION - 3-MODEL ENSEMBLE)
# ===========================================================================
def predict(test: pl.DataFrame) -> float:
    
    # Define constants locally for the predict function scope
    BEST_W_XGB = 4.183022496349415
    BEST_W_LGB = 0.29640061148676144
    BEST_W_CAT = 1.9283230458964578
    BEST_W_LOGREG = 0.0 
    ML_CONF_FACTOR = 6.6208527134892385 
    
    # Check if all models are present (LogReg is needed for prediction, even if weighted 0)
    if xgb_model is None or lgb_model is None or cbt_model is None or logreg_model is None or scaler is None or not FINAL_FEATURE_COLS:
        return 0.0

    date_id = None
    try:
        date_id = int(test.select("date_id").to_series().item())
    except:
        pass
    
    try:
        test_engineered = create_features(test)
        X_test = test_engineered.select(FINAL_FEATURE_COLS).fill_null(0).to_pandas()
        
        X_clean = X_test.loc[:, FINAL_FEATURE_COLS]
        X_clean = X_clean.fillna(0).replace([np.inf, -np.inf], 0) 

        X_scaled = scaler.transform(X_clean.values)
        X_lgb = pd.DataFrame(X_scaled, columns=FINAL_FEATURE_COLS)
        dtest = xgb.DMatrix(X_scaled) 

        # Predict probabilities (4 models)
        prob_xgb = xgb_model.predict(dtest)[0] 
        prob_lgb = lgb_model.predict_proba(X_lgb)[:, 1][0]
        prob_cbt = cbt_model.predict_proba(X_clean)[:, 1][0] 
        prob_logreg = logreg_model.predict_proba(X_lgb)[:, 1][0] 
        
        # Ensemble and Risk Adjustment (using 3 positive weights)
        total_w = BEST_W_XGB + BEST_W_LGB + BEST_W_CAT + BEST_W_LOGREG
        # This average calculation must be right: W_LOGREG is 0.0.
        avg_prob = (BEST_W_XGB * prob_xgb + BEST_W_LGB * prob_lgb + BEST_W_CAT * prob_cbt + BEST_W_LOGREG * prob_logreg) / total_w

        confidence = 2 * abs(avg_prob - 0.5)
        position = np.clip(confidence * ML_CONF_FACTOR, 0, 2)

        return float(position)

    except Exception as e:
        logger.warning(f"ML Error (date_id: {date_id}): {e}")
        return 0.0


# ===========================================================================
# SERVER
# ===========================================================================
logger.info("Starting server")
import kaggle_evaluation.default_inference_server
inference_server = kaggle_evaluation.default_inference_server.DefaultInferenceServer(predict)

if os.getenv('KAGGLE_IS_COMPETITION_RERUN'):
    inference_server.serve()
else:
    inference_server.run_local_gateway((str(DATA_PATH),))

logger.info("✅ Complete. ")

2025-12-05 15:03:03,226 - Loading data and splitting for validation...
2025-12-05 15:03:03,814 - ✅ Oracle Dictionary ready. Binary Target created.
2025-12-05 15:03:03,815 - Using OPTUNA BEST ENSEMBLE (Trial 3258), Weights: XGB:4.18, LGB:0.30, CAT:1.93
2025-12-05 15:03:03,816 - Using OPTUNA BEST ENSEMBLE (Trial 3258), Risk Factor: 6.6209
2025-12-05 15:03:03,817 - Training ensemble (Expanded Feature Set)...
2025-12-05 15:03:09,676 - Features: 1187 (Restored Optimized Set)
2025-12-05 15:03:32,677 - XGBoost trained (Stopped at 18 trees).
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated

In [2]:
import pandas as pd
import numpy as np
import os

submission_path = '/kaggle/working/submission.parquet'

## 📊 SUBMISSION FILE VALIDATION

# Check if the file was successfully created
if not os.path.exists(submission_path):
    print(f"Validation Error: Submission file not found at {submission_path}")
    print("NOTE: This file is created during the 'Running local gateway for testing...' step.")
else:
    try:
        # Read the submission file
        df_sub = pd.read_parquet(submission_path)

        # --- Validation Checks ---

        # 1. Identify the prediction column (the single float column)
        float_cols = df_sub.select_dtypes(include=[np.float64]).columns

        if len(float_cols) == 1:
            prediction_col_name = float_cols[0]
            print(f"Column Check: Found single prediction column named '{prediction_col_name}'.")
        else:
            prediction_col_name = 'allocation' # Use standard name for range check fallback
            print(f"Column Check: Found {len(float_cols)} float columns. Using '{prediction_col_name}' for range check.")

        # 2. Check allocation range (0.0 to 2.0)
        if prediction_col_name in df_sub.columns:
            min_val = df_sub[prediction_col_name].min()
            max_val = df_sub[prediction_col_name].max()

            # Check if all values are between 0.0 and 2.0 (inclusive)
            if min_val >= 0.0 and max_val <= 2.0:
                range_check = "PASS"
            else:
                range_check = f"FAIL (Min: {min_val:.4f}, Max: {max_val:.4f})"

            print(f"Allocation Range Check (0.0 to 2.0): {range_check}")

        # 3. Display the file info
        print("\nFirst 5 Rows of Submission:")
        print(df_sub.head())

        print("\nSubmission Info:")
        df_sub.info()

    except Exception as e:
        print(f"Validation Error: Could not read or process the Parquet file. Error: {e}")

Column Check: Found single prediction column named 'prediction'.
Allocation Range Check (0.0 to 2.0): PASS

First 5 Rows of Submission:
   date_id  prediction
0     8980    0.056691
1     8981    0.256202
2     8982    0.917846
3     8983    0.186293
4     8984    0.061686

Submission Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   date_id     10 non-null     int64  
 1   prediction  10 non-null     float64
dtypes: float64(1), int64(1)
memory usage: 292.0 bytes
